### Advanced in silico drug design workshop. Olomouc, 21-25 January, 2019.
### Deep Leaning Tutorial: Multi-Layer Perceptron with Keras

Dr Thomas Evangelidis

Research Scientist

IOCB - Institute of Organic Chemistry and Biochemistry of the Czech Academy of Sciences
Prague, Czech Republic

  & 
CEITEC - Central European Institute of Technology
Brno, Czech Republic 

email: tevang3@gmail.com

website: https://sites.google.com/site/thomasevangelidishomepage/


##Objectives:

In this tutorial you will learn how to construct a simple Multi-Layer Perceptron model with Keras. Specifically you will learn to:
* Create and add layers including weight initialization and activation.
* Compile models including optimization method, loss function and metrics.
* Fit models include epochs and batch size.
* Model predictions.
* Summarize the model.
* Train an initial model on large relevant data and transfer the hidden layers of that model to a new one, which will be training with fewer focused data (Transfer LEarning).

In [45]:
from keras.wrappers.scikit_learn import KerasRegressor, KerasClassifier
from keras.callbacks import EarlyStopping
from rdkit.Chem import AllChem, SDMolSupplier
from rdkit import DataStructs
import numpy as np
from keras.models import Sequential, Input, Model
from keras.layers import Dense
from scipy.stats import kendalltau, pearsonr
from sklearn.metrics import mean_squared_error, roc_auc_score
from sklearn.model_selection import cross_validate, KFold

#### Reading molecules and activity from SDF

In [56]:
fname = "data/cdk2.sdf"

mols = []
y = []
for mol in SDMolSupplier(fname):
    if mol is not None:
        mols.append(mol)
        y.append(float(mol.GetProp("pIC50")))

#### Calculate descriptors (fingerprints) and convert them into numpy array

In [57]:
# generate binary Morgan fingerprint with radius 2
fp = [AllChem.GetMorganFingerprintAsBitVect(m, 2) for m in mols]

In [58]:
def rdkit_numpy_convert(fp):
    output = []
    for f in fp:
        arr = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(f, arr)
        output.append(arr)
    return np.asarray(output)

In [59]:
x = rdkit_numpy_convert(fp)

In [60]:
# fix random seed for reproducibility
seed = 2019
np.random.seed(seed)
mol_num, feat_num = x.shape
print("# molecules for training = %i, # of features = %i\n" % (mol_num, feat_num))

# molecules for training = 436, # of features = 2048



#### Define a function to create a simple feed forward network with one fully connected hidden layer or 300 neurons. The network uses the rectifier activation function for the hidden layer. No activation function is used for the output layer because it is a regression problem and we are interested in predicting numerical values directly without transform. The ADAM algorithm is employed to optimize the loss function.

In [61]:
# create model
def MLP1(feat_num, loss):
    net = Sequential()
    net.add(Dense(300, input_dim=feat_num, kernel_initializer='normal', activation='relu'))
    net.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    net.compile(loss=loss, optimizer='adam')
    return net

#### Print summary of layers and trainable parameters.

In [62]:
MLP1(feat_num, 'mean_squared_error').summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_104 (Dense)            (None, 300)               614700    
_________________________________________________________________
dense_105 (Dense)            (None, 1)                 301       
Total params: 615,001
Trainable params: 615,001
Non-trainable params: 0
_________________________________________________________________


#### Evaluate model with Keras wrapper of scikit-learn (faster and easier), using Mean Squared Error as the loss function, 300 training epochs and batch size 1/8 of the training set.

In [63]:
estimator = KerasRegressor(build_fn=MLP1, 
                           feat_num=feat_num, 
                           loss='mean_squared_error', 
                           epochs=300, 
                           batch_size=int(x.shape[0]/8), 
                           verbose=0)

#### Define our own evaluation metrics for the model: 1) Kendall's tau (ranking correlation), 2) Pearson's R (correlation), 3) Mean Squared Error. The evaluation will be done with 5-fold cross-validation.

In [65]:
def kendalls_tau(estimator, X, y):
    preds = estimator.predict(X)
    t = kendalltau(preds, y)[0]
    return t

def pearsons_r(estimator, X, y):
    preds = estimator.predict(X)
    r = pearsonr(preds, y)[0]
    return r

def MSE(estimator, X, y):
    preds = estimator.predict(X)
    mse = mean_squared_error(preds, y)
    return mse
    
    
scoring = {'tau': kendalls_tau, 'R':pearsons_r, 'MSE':MSE}
kfold = KFold(n_splits=5, random_state=seed)
scores = cross_validate(estimator, x, y, scoring=scoring, cv=kfold, return_train_score=False)
print scores
print("\nResults: average tau=%f+-%f, average R=%f+-%f, average MSE=%f+-%f\n" % 
      (scores['test_tau'].mean(), scores['test_tau'].std(),
       scores['test_R'].mean(), scores['test_R'].std(),
       scores['test_MSE'].mean(), scores['test_MSE'].std()))

{'score_time': array([0.27829313, 0.26521516, 0.30018497, 0.28733611, 0.28519821]), 'test_MSE': array([1.56494677, 2.08746199, 2.82407414, 0.83487712, 0.93305331]), 'fit_time': array([22.1518569 , 21.32536888, 20.33078098, 20.02604198, 20.24820685]), 'test_R': array([0.60387652, 0.45365342, 0.52166595, 0.83160936, 0.85404569]), 'test_tau': array([0.40732997, 0.27297084, 0.25495331, 0.58509993, 0.6116522 ])}

Results: average tau=0.426401+-0.150201, average R=0.652970+-0.162309, average MSE=1.648883+-0.742360



#### Running this code gives us an estimate of the model’s performance on the problem for unseen data. The result reports the average and standard deviation of each metric across all 5 folds of the cross validation evaluation.

#### Now let's try Absolute Mean Error as a loss function.

In [13]:
estimator = KerasRegressor(build_fn=MLP1, 
                           feat_num=feat_num, 
                           loss='mean_absolute_error', 
                           epochs=300, 
                           batch_size=int(x.shape[0]/8),
                           verbose=0)

In [14]:
scores = cross_validate(estimator, x, y, scoring=scoring, cv=kfold, return_train_score=False)
print scores
print("\nResults: average tau=%f+-%f, average R=%f+-%f, average MSE=%f+-%f\n" % 
      (scores['test_tau'].mean(), scores['test_tau'].std(),
       scores['test_R'].mean(), scores['test_R'].std(),
       scores['test_MSE'].mean(), scores['test_MSE'].std()))

{'score_time': array([0.03722405, 0.04201698, 0.05009317, 0.0542469 , 0.055197  ]), 'test_MSE': array([1.42742991, 1.8259697 , 2.20100964, 0.8656745 , 0.8802869 ]), 'fit_time': array([22.58931589, 22.98770189, 23.99517798, 23.59210014, 22.48769593]), 'test_R': array([0.60599303, 0.51725032, 0.60525591, 0.82568417, 0.86391249]), 'test_tau': array([0.40104724, 0.32118944, 0.33132128, 0.60386172, 0.62990246])}

Results: average tau=0.457464+-0.133291, average R=0.683619+-0.136039, average MSE=1.440074+-0.523716



#### We see a subtle performance increase.

#### Now let's add an extra hidden layer to the network with 200 neurons, to see if the performance improves further.

In [71]:
# create model
def MLP2(feat_num, loss):
    net = Sequential()
    net.add(Dense(300, input_dim=feat_num, kernel_initializer='normal', activation='relu'))
    net.add(Dense(200, input_dim=feat_num, kernel_initializer='normal', activation='relu'))
    net.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    net.compile(loss=loss, optimizer='adam')
    return net

#### Print summary of layers and trainable parameters.

In [72]:
MLP2(feat_num, 'mean_absolute_error').summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_116 (Dense)            (None, 300)               614700    
_________________________________________________________________
dense_117 (Dense)            (None, 200)               60200     
_________________________________________________________________
dense_118 (Dense)            (None, 1)                 201       
Total params: 675,101
Trainable params: 675,101
Non-trainable params: 0
_________________________________________________________________


#### We increase the training epochs to 500 because the addition of an extra layer increased the trainable variables.

In [73]:
estimator = KerasRegressor(build_fn=MLP2, 
                           feat_num=feat_num, 
                           loss='mean_absolute_error', 
                           epochs=500,
                           batch_size=int(x.shape[0]/8), 
                           verbose=0)

In [18]:
scores = cross_validate(estimator, x, y, scoring=scoring, cv=kfold, return_train_score=False)
print scores
print("\nResults: average tau=%f+-%f, average R=%f+-%f, average MSE=%f+-%f\n" % 
      (scores['test_tau'].mean(), scores['test_tau'].std(),
       scores['test_R'].mean(), scores['test_R'].std(),
       scores['test_MSE'].mean(), scores['test_MSE'].std()))

{'score_time': array([0.08743596, 0.1128571 , 0.10812306, 0.08129883, 0.08918095]), 'test_MSE': array([1.31824432, 1.72178322, 2.78364743, 0.86386301, 0.88395526]), 'fit_time': array([49.73342609, 44.79159188, 37.4734621 , 34.84491301, 30.94613099]), 'test_R': array([0.62119832, 0.53859465, 0.52101726, 0.82476026, 0.85846916]), 'test_tau': array([0.41151845, 0.34529874, 0.24785001, 0.60118147, 0.61648315])}

Results: average tau=0.444466+-0.144033, average R=0.672808+-0.142321, average MSE=1.514299+-0.708962



#### We don't see statistically significant improvement because our training set is small (436 molecules).

# Transfer Learning

#### We will use all binding assays for CDK2 from CHEMBL (1519 molecules) to train a network and then we will transfer its hidden layer to a new network which we shall train with the smaller training set that we have been using so far.

#### The following block of code is just for data preparation,  you can go hrough it very fast.

In [21]:
# LOAD TRAINING DATA FOR TRANSFER LEARNING
fname = "data/cdk2_large.sdf"

mols = []
molnames = []
for mol in SDMolSupplier(fname):
    if mol is not None:
        molname = mol.GetProp("_Name")
        if not molname in molnames:
            molnames.append(molname)
            mols.append(mol)

In [22]:
molname2pK_dict = {}
with open("data/cdk2_pK.dat", 'r') as f:
    for line in f:
        molname, pK = line.split()
        if not molname in molname2pK_dict.keys():
            molname2pK_dict[molname] = float(pK)

In [ ]:
molnames1 = set(molnames)
molnames2 = set(molname2pK_dict.keys())
common_molnames = molnames1.intersection(molnames2)
y_transf = [molname2pK_dict[molname] for molname in molnames if molname in common_molnames]

#### Generate binary Morgan fingerprint with radius 2 as feature vectors for training.

In [24]:
fp = [AllChem.GetMorganFingerprintAsBitVect(m, 2) for m in mols if m.GetProp("_Name") in common_molnames]
x_transf = rdkit_numpy_convert(fp)

In [25]:
mol_num, feat_num = x_transf.shape
print("# molecules for transfer training = %i, # of features = %i\n" % (mol_num, feat_num))

# molecules for transfer training = 1519, # of features = 2048



#### We train a network with one fully connected hidden layer of 300 neurons, like in our first example.

In [ ]:
net = MLP1(feat_num=feat_num, 
           loss='mean_absolute_error')
net.fit(x_transf, 
        y_transf,
        epochs=300, 
        batch_size=int(x_transf.shape[0]/8), 
        verbose=0)

#### Below we create a function that transfers hiddel layers (up to index 'idx', starting from 0) to a new network. 'lhl_sizes' is a tuple defining the number of neurons in each hidden layer, e.g. (200,100) means two hidden layers with 200 and 100 neurons respectivelly.

In [74]:
def transf_MLP(feat_num, idx, lhl_sizes, loss='mean_absolute_error'):
    global net  # net is a networks and cannot be pickled! Therefore it cannot be an input argument for cross_validate() to work!
    inp = Input(shape=(feat_num,))
    shared_layer = net.layers[0]
    shared_layer.trainable = False  # deactivate training in all re-used layers of MLP1
    out_tensor = shared_layer(inp)
    # idx = 1  # index of desired layer
    for i in range(1,idx+1):
        shared_layer = net.layers[i]    # deactivate training in all re-used layers of MLP1
        shared_layer.trainable = False  # deactivate training in all re-used layers of MLP1
        out_tensor = shared_layer(out_tensor)
    # Here add all the new layers
    for l_size in lhl_sizes:
        out_tensor = Dense(l_size, kernel_initializer='normal', activation='relu')(out_tensor)
    # Close the network
    out_tensor = Dense(1, kernel_initializer='normal')(out_tensor)
    # Create the model
    transf_model = Model(inp, out_tensor)
    transf_model.compile(loss=loss, optimizer='adam')
    return transf_model

In [75]:
estimator = KerasRegressor(build_fn=transf_MLP,
                           feat_num=feat_num,
                           idx=0,
                           lhl_sizes=(300, 200),
                           loss='mean_absolute_error',
                           epochs=300,
                           batch_size=int(x.shape[0]/8),
                           verbose=0)

#### Measure the performance of the new hybrid network on our initial small dataset with 5-fold cross-validation.

In [76]:
scores = cross_validate(estimator, x, y, scoring=scoring, cv=kfold, return_train_score=False)
print scores
print("\nResults: average tau=%f+-%f, average R=%f+-%f, average MSE=%f+-%f\n" %
      (scores['test_tau'].mean(), scores['test_tau'].std(),
       scores['test_R'].mean(), scores['test_R'].std(),
       scores['test_MSE'].mean(), scores['test_MSE'].std()))

{'score_time': array([0.29660296, 0.2901659 , 0.30451417, 0.30216289, 0.32080102]), 'test_MSE': array([1.15663282, 1.28708681, 0.70786904, 1.21018291, 1.19200641]), 'fit_time': array([7.6896379 , 8.12507606, 7.61119699, 7.39933205, 7.80200386]), 'test_R': array([0.68691322, 0.81303906, 0.87155735, 0.80353104, 0.85249823]), 'test_tau': array([0.48848183, 0.62335932, 0.62241412, 0.64245741, 0.62346119])}

Results: average tau=0.600035+-0.056280, average R=0.805508+-0.064337, average MSE=1.110756+-0.205914



#### We see an impressive performance gain!

In [78]:
scores = cross_validate(estimator, x, y, scoring=scoring, cv=kfold, return_train_score=False,)
print scores
print("\nResults: average tau=%f+-%f, average R=%f+-%f, average MSE=%f+-%f\n" %
      (scores['test_tau'].mean(), scores['test_tau'].std(),
       scores['test_R'].mean(), scores['test_R'].std(),
       scores['test_MSE'].mean(), scores['test_MSE'].std()))

{'score_time': array([0.35903096, 0.35371709, 0.36846399, 0.36452699, 0.37568092]), 'test_MSE': array([1.16594237, 1.34469876, 0.78113445, 1.14604242, 1.09889925]), 'fit_time': array([9.7914331 , 7.95971394, 8.05258393, 8.59206796, 8.59934092]), 'test_R': array([0.69472628, 0.79244967, 0.8679898 , 0.81822004, 0.8420003 ]), 'test_tau': array([0.50000015, 0.60782221, 0.61140077, 0.65371449, 0.59554902])}

Results: average tau=0.593697+-0.050789, average R=0.803077+-0.059684, average MSE=1.107343+-0.183168



#### NOTE:
Transfer learning does not always improve the performance. In this case we used a larger set of compounds binding to the same receptor (CDK2) to pretrain an network and transfer it's hidden layer to another one. If we have done the same but with compounds binding to CDK1 (59% sequence identity with CDK2) then the performance would have been worse. So be caucious where you apply transfer learning and which training data you use!
As a home work you can apply the same procedure by instead of "data/cdk2_large.sdf" and "data/cdk2_pK.dat", to use "data/cdk1_large.sdf" and "data/cdk1_pK.dat".

## Train a Classifier network instead of a Regressor.

#### The difference with our Regressor MLP1 is that the output layer contains a single neuron and uses the sigmoid activation function in order to produce a probability output in the range of 0 to 1 that can easily and automatically be converted to class values.

In [ ]:
# create a Classifier
def MLP3(feat_num, loss='binary_crossentropy'):
    net = Sequential()
    net.add(Dense(300, input_dim=feat_num, kernel_initializer='normal', activation='relu'))
    net.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    net.compile(optimizer='rmsprop', loss=loss, metrics=['accuracy'])
    # Compile model
    net.compile(loss=loss, optimizer='adam')
    return net

#### Load and prepare the Blood Brain Barrier permeability data for classification.

In [82]:
fname = "data/logBB.sdf"

mols = []
y = []
for mol in SDMolSupplier(fname):
    if mol is not None:
        mols.append(mol)
        y.append(float(mol.GetProp("logBB_class")))

#### Generate binary Morgan fingerprint with radius 2 for training.

In [83]:
fp = [AllChem.GetMorganFingerprintAsBitVect(m, 2) for m in mols]

In [84]:
x = rdkit_numpy_convert(fp)

In [85]:
mol_num, feat_num = x.shape
print("# molecules for training = %i, # of features = %i\n" % (mol_num, feat_num))

# molecules for training = 321, # of features = 2048



#### Print summary of layers and trainable parameters.

In [86]:
MLP3(feat_num, 'binary_crossentropy').summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_166 (Dense)            (None, 300)               614700    
_________________________________________________________________
dense_167 (Dense)            (None, 1)                 301       
Total params: 615,001
Trainable params: 615,001
Non-trainable params: 0
_________________________________________________________________


In [92]:
estimator = KerasClassifier(build_fn=MLP3, 
                           feat_num=feat_num, 
                           loss='binary_crossentropy', 
                           epochs=100,  # ~300 is the optimum value, but we use 1000 to see the effect of overfitting
                           batch_size=int(x.shape[0]/8), 
                           verbose=0)

#### Use this time a classification metric to score the predictions, the area under the Receiver Operating Characteristic Curve (AUC-ROC).

In [93]:
def AUC_ROC(estimator, X, y):
    preds = estimator.predict(X)
    auc = roc_auc_score(preds, y)
    return auc

scoring = {'roc': AUC_ROC}
kfold = KFold(n_splits=5, random_state=seed)
scores = cross_validate(estimator, x, y, scoring=scoring, cv=kfold, return_train_score=False)
print scores
print("\nResults: average AUC-ROC=%f+-%f\n" % 
      (scores['test_roc'].mean(), scores['test_roc'].std()))

{'score_time': array([0.48979092, 0.50375891, 0.48540902, 0.5265758 , 0.53580093]), 'test_roc': array([0.69588745, 0.7017017 , 0.81118494, 0.78615385, 0.61862745]), 'fit_time': array([7.49084401, 7.00696516, 6.69053817, 7.84793019, 7.08765721])}

Results: average AUC-ROC=0.722711+-0.069061



#### The MLP classifier had a realative good performance. Compare with with ML model performance from the QSAR tutorial.

#### Let's use early stopping to see if the performance improves even further.

In [89]:
scores = cross_validate(estimator, x, y, scoring=scoring, cv=kfold, return_train_score=False,
                        fit_params={'callbacks': [EarlyStopping(patience=3)]})
print scores
print("\nResults: average AUC-ROC=%f+-%f\n" % 
      (scores['test_roc'].mean(), scores['test_roc'].std()))

{'score_time': array([0.44591999, 0.45287895, 0.46037412, 0.45666695, 0.46215296]), 'test_roc': array([0.69622426, 0.66966967, 0.81118494, 0.79757085, 0.61862745]), 'fit_time': array([23.10912299, 21.18485212, 21.2553308 , 21.11846805, 20.74858093])}

Results: average AUC-ROC=0.718655+-0.074428



## Try to recreate each of the network architectures that you created with Keras using the following online tool:

#### http://playground.tensorflow.org

#### We don't see statistically significant change, probably because the training set is small (321 molecules). In real case problems you have thousands of training samples and there the effects of overfitting are more evident.